In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [8]:
!kaggle competitions download -c fake-news

 95% 44.0M/46.5M [00:02<00:00, 32.8MB/s]
100% 46.5M/46.5M [00:02<00:00, 20.6MB/s]


In [9]:
import zipfile
zip_ref = zipfile.ZipFile('/content/fake-news.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [16]:
import pandas as pd

In [25]:
df = pd.read_csv("/content/train.csv")

In [26]:
df.shape

(20800, 5)

In [27]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [28]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [29]:
df = df.dropna()

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18285 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18285 non-null  int64 
 1   title   18285 non-null  object
 2   author  18285 non-null  object
 3   text    18285 non-null  object
 4   label   18285 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 857.1+ KB


In [31]:
x = df.drop('label',axis=1)

In [32]:
y = df['label']

In [33]:
import tensorflow as tf

In [34]:
tf.__version__

'2.12.0'

In [36]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [37]:
voc_size = 5000

In [38]:
messages=x.copy()

In [39]:
messages.reset_index(inplace=True)

In [40]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18285 entries, 0 to 18284
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   18285 non-null  int64 
 1   id      18285 non-null  int64 
 2   title   18285 non-null  object
 3   author  18285 non-null  object
 4   text    18285 non-null  object
dtypes: int64(2), object(3)
memory usage: 714.4+ KB


In [41]:
import nltk
import re
from nltk.corpus import stopwords

In [42]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [44]:
## Data Preprocessing

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [46]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
onehot_repr

[[191, 3815, 1745, 287, 1381, 1008, 4579, 4971, 1010, 4023],
 [3977, 3591, 4709, 1222, 3834, 3992, 4505],
 [383, 1430, 2224, 2097],
 [2578, 4411, 3286, 3360, 3874, 4935],
 [1483, 3834, 3918, 1143, 2584, 2813, 3834, 2532, 1874, 2018],
 [4755,
  2698,
  1497,
  3036,
  587,
  4745,
  4397,
  2473,
  4715,
  3303,
  992,
  2305,
  899,
  3649,
  4505],
 [700, 4712, 2221, 4298, 1149, 216, 3450, 336, 778, 3902, 2244],
 [2569, 613, 552, 1665, 1895, 4458, 4745, 4625, 778, 3902, 2244],
 [4138, 788, 1996, 4698, 3202, 2913, 598, 3701, 4745, 2428],
 [4720, 3034, 4100, 2004, 1545, 4194, 4981, 1093],
 [354, 63, 535, 3034, 2431, 955, 3912, 2792, 3766, 1957, 2973],
 [3360, 3093, 1381, 2913, 4745, 1895],
 [3747, 4334, 253, 539, 3689, 3742, 3810, 4732, 789],
 [3901, 2510, 2229, 4468, 1556, 405, 3522, 778, 3902, 2244],
 [3892, 48, 667, 4934, 4237, 778, 3902, 2244],
 [1720, 4545, 1785, 200, 175, 3522, 1866, 3496, 4617, 4333],
 [4570, 4688, 3591],
 [4003, 4320, 3771, 803, 4745, 3317, 157, 4505],
 [3353, 5

In [47]:
sent_length = 20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 4971, 1010, 4023],
       [   0,    0,    0, ..., 3834, 3992, 4505],
       [   0,    0,    0, ..., 1430, 2224, 2097],
       ...,
       [   0,    0,    0, ...,  778, 3902, 2244],
       [   0,    0,    0, ..., 1337, 3897, 4269],
       [   0,    0,    0, ..., 1738, 1870, 3312]], dtype=int32)

In [48]:
from keras.api._v2.keras import activations
## Creating Model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [49]:
import numpy as np
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [50]:
x_final.shape,y_final.shape

((18285, 20), (18285,))

In [51]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final,test_size=0.2,random_state=123)

In [52]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
229/229 [==============================] - 14s 48ms/step - loss: 0.3118 - accuracy: 0.8511 - val_loss: 0.1756 - val_accuracy: 0.9289
Epoch 2/10
229/229 [==============================] - 11s 47ms/step - loss: 0.1397 - accuracy: 0.9449 - val_loss: 0.1793 - val_accuracy: 0.9303
Epoch 3/10
229/229 [==============================] - 11s 49ms/step - loss: 0.1035 - accuracy: 0.9607 - val_loss: 0.2008 - val_accuracy: 0.9243
Epoch 4/10
229/229 [==============================] - 9s 41ms/step - loss: 0.0836 - accuracy: 0.9702 - val_loss: 0.2199 - val_accuracy: 0.9150
Epoch 5/10
229/229 [==============================] - 17s 75ms/step - loss: 0.0649 - accuracy: 0.9762 - val_loss: 0.2408 - val_accuracy: 0.9158
Epoch 6/10
229/229 [==============================] - 11s 47ms/step - loss: 0.0492 - accuracy: 0.9822 - val_loss: 0.2937 - val_accuracy: 0.9218
Epoch 7/10
229/229 [==============================] - 9s 39ms/step - loss: 0.0320 - accuracy: 0.9898 - val_loss: 0.3180 - val_accuracy: 0

In [53]:
y_pred = model.predict(x_test)

115/115 [==============================] - 3s 14ms/step


In [54]:
y_pred = np.where(y_pred > 0.5, 1, 0)

In [59]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
confusion_matrix(y_test,y_pred)

array([[1917,  136],
       [ 183, 1421]])

In [58]:
accuracy_score(y_test,y_pred)

0.9127700300793

In [62]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92      2053
           1       0.91      0.89      0.90      1604

    accuracy                           0.91      3657
   macro avg       0.91      0.91      0.91      3657
weighted avg       0.91      0.91      0.91      3657

